## Introduction

In this jupyter notebook, we will evaluate a pretrained `torchreid` model called `osnet_ain_x1_0`. This particular model is trained on different Re-Id data sets (Market1501, DukeMTMC-ReID, and MSMT17).

In order to run the notebook, first mount your google drive and upload the following files:

- `osnet_ain_x1_0` model weights (look under multi-source domain generatlization and download the `MS+D+C->M` model version using this [link](https://kaiyangzhou.github.io/deep-person-reid/MODEL_ZOO.html))
- 'market1501` dataset which can be downloaded from kaggle using this [link](https://www.kaggle.com/datasets/pengcw1/market-1501)




For now download these datasets and upload them to your google drive.

## Data paths

In [1]:
# change your root path
import os

path = "/content/drive/MyDrive"  # root
os.chdir(path)
os.listdir(path)

['NeurIPS 2022.gsheet',
 'Meet Recordings',
 'poverty-mapping',
 'Colab Notebooks',
 'Poverty Mapping: Results.gdoc',
 'poverty-mapping-literature',
 'Kim, Moses - Regularization Self-Evaluation',
 'vhm-ysmb-kkf - Aug 24, 2022.gjam',
 'FunML',
 'poverty-mapping-data',
 'ML Walk-In Clinic.gslides',
 'ML Walk-In Clinic: Consulting Best Practices and Standards.gdoc',
 'ML Walk-In Clinic',
 'Untitled document.gdoc',
 'playground',
 'big-ml-team',
 'Poverty Mapping in SEA: Rollout Challenges.gdoc',
 'ML_20220311_December.csv.zip',
 'psa',
 'powerpoint',
 'Resume: Moses Kim.gdoc',
 'Learning',
 'Market-1501-v15.09.15.zip',
 'Person_reID_baseline_pytorch',
 'reid',
 'deep-person-reid',
 'reid-output']

In [11]:
parent_dir = "Learning"  # couldn't think about a better name

# create the data directory reid-data
dir_name = "reid-data"
dir_path = os.path.join(path, parent_dir, dir_name)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# create the model directory reid-models
models_path = os.path.join(path, parent_dir, "reid-models")
if not os.path.exists(models_path):
    os.makedirs(models_path)


Once you have run the two cells above, upload both model weights and datasets following the directory structure below.

```
<root>
└───<parent-directory>
    │   ...
    │   ...     
    └───reid-data
    │   └───market1501
    │       └───Market-1501-v15.09.15
    │       │   └───bounding_box_test
    │       │   └───bounding_box_train
    │       │   └───query
    └───reid-models
        │   osnet_ain_ms_d_c.pth.tar
        │   ...
```

## `torchreid` installation

In [3]:
# don't download twice
repo_dir_path = os.path.join(path, "deep-person-reid")

if not os.path.exists(repo_dir_path):
    repo_path = "https://github.com/KaiyangZhou/deep-person-reid.git"
    os.system(f"git clone {repo_path}")

In [4]:
# cd into repo and install requirements 

%cd /content/drive/MyDrive/deep-person-reid/
!pip install -r requirements.txt

/content/drive/MyDrive/deep-person-reid
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.7 MB 4.1 MB/s 
     |████████████████████████████████| 57 kB 5.4 MB/s 
     |████████████████████████████████| 190 kB 60.4 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 41 kB 247 kB/s 


In [5]:
# install torch and torchvision
# (should be installed by defaults)

!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# finally, install torchreid package
!python setup.py develop

running develop
running egg_info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'torchreid.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.8/torchreid/metrics/rank_cylib/rank_cy.cpython-38-x86_64-linux-gnu.so -> torchreid/metrics/rank_cylib
Creating /usr/local/lib/python3.8/dist-packages/torchreid.egg-link (link to .)
Adding torchreid 1.4.0 to easy-install.pth file

Installed /content/drive/MyDrive/deep-person-reid
Processing dependencies for torchreid==1.4.0
Searching for imageio==2.9.0
Best match: imageio 2.9.0
Adding imageio 2.9.0 to easy-install.pth file
Installing imageio_download_bin script to /usr/local/bin
Installing imageio_remove_bin script to /usr/local/bin

Using /usr/local/lib/python3.8/dist-packages
Searching for isort==4.3.21

In [7]:
# check
import torchreid

In [8]:
# now change back to root
# or wherever you wish to save output

output_name = "reid-output"
output_path = os.path.join(
    path,
    parent_dir,
    output_name
)

if not os.path.exists(output_path):
    os.makedirs(output_path)

os.chdir(output_path)

## Load Model

In [9]:
from torchreid import models, utils, engine

In [12]:
model_wt_filename = "osnet_ain_ms_d_c.pth.tar"

weight_path = os.path.join(
    models_path,
    model_wt_filename
)

# backbone
num_classes = 1501 + 1467 + 1404  # unique ids from Market1501, MSMT17, and DukeMTMC-reID datasets
model = models.build_model(
    name='osnet_ain_x1_0',
    num_classes=num_classes,  # the only way is to have access to the data
    loss='softmax',
    pretrained=True,
    use_gpu=False   # colab might have usage limits; was able to train using gpu though
)

# finally, load weights
utils.load_pretrained_weights(model, weight_path)

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Successfully loaded pretrained weights from "/content/drive/MyDrive/Learning/reid-models/osnet_ain_ms_d_c.pth.tar"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


## Evaluate Model

In [16]:
# create manager object market1501 data
market_datamanager = torchreid.data.ImageDataManager(
    root=dir_path,
    sources='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    combineall=True
)

# create engine to test
# first need to create optimizer
# TBH, I am not sure why the code requires an optimizer when running eval
# there are many inconsistencies and inconveniences like this
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
engine = torchreid.engine.ImageSoftmaxEngine(
    market_datamanager, model, optimizer
)

engine.run(
    max_epoch=60,
    save_dir='log/osnet-softmax-market1501',
    print_freq=10,
    test_only=True,
    visrank=True
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |  1501 |    29419 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |  1501 |    29419 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 1501
  # source images   : 29419
  # source cameras  : 6
  target            : ['market1501']
  *****************************************


##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Speed: 39.9016 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 43.8%
CMC curve
Rank-1  : 72.4%
Rank-5  : 85.9%
Rank-10 : 89.8%
Rank-20 : 93.1%
# query: 3368
# gal

In [18]:
# single sample

# get test loader from datamanager
market_data_loader = market_datamanager.train_loader

sample_batch = next(iter(market_data_loader))['img']
sample = sample_batch[0]

sample

tensor([[[-0.3027, -0.3369, -0.3883,  ..., -0.1143, -0.1657, -0.1999],
         [-0.2856, -0.3198, -0.3712,  ..., -0.0116, -0.0458, -0.0801],
         [-0.2684, -0.3027, -0.3541,  ...,  0.1939,  0.1768,  0.1426],
         ...,
         [ 0.4851,  0.4679,  0.4508,  ..., -0.3541, -0.3369, -0.3198],
         [ 0.4851,  0.4851,  0.4679,  ...,  0.1768,  0.2111,  0.2282],
         [ 0.4851,  0.4851,  0.4679,  ...,  0.4508,  0.4851,  0.5022]],

        [[-0.3025, -0.3375, -0.3901,  ...,  0.0651,  0.0126, -0.0224],
         [-0.2850, -0.3200, -0.3725,  ...,  0.1877,  0.1352,  0.1176],
         [-0.2675, -0.3025, -0.3550,  ...,  0.4153,  0.3978,  0.3803],
         ...,
         [ 0.4328,  0.4153,  0.3978,  ..., -0.4076, -0.3901, -0.3725],
         [ 0.4328,  0.4328,  0.4153,  ...,  0.1527,  0.1877,  0.2052],
         [ 0.4328,  0.4328,  0.4153,  ...,  0.4328,  0.4678,  0.4853]],

        [[-0.1835, -0.2184, -0.2707,  ..., -0.2532, -0.3055, -0.3404],
         [-0.1661, -0.2010, -0.2532,  ..., -0

In [23]:
# let's check out the embedding (features) for the sample
import torch

sample_emb = model(torch.unsqueeze(sample, 0))
sample_emb.shape

torch.Size([1, 512])